In [3]:
pip install pandas

     --------------------------------------- 11.2/11.2 MB 32.8 MB/s eta 0:00:00
     ------------------------------------- 502.3/502.3 kB 10.5 MB/s eta 0:00:00
     -------------------------------------- 341.8/341.8 kB 7.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: C:\Users\rs49742\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [23]:
import sys
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.results as res
from biogeme.exceptions import biogemeError
from biogeme.expressions import Beta, Variable, bioDraws, MonteCarlo, log, exp, PanelLikelihoodTrajectory
import biogeme.distributions as dist
import matplotlib.pyplot as plt


In [5]:
# Read the data
micro_pool_socio = pd.read_csv('Data/micro_pool_socio_bio2up.csv')

In [6]:
# Micromobility Trips 0-5 Miles
micro0to5 = (((
# 0-2 miles first choice
micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
# 0-2 miles second choice
    # scooter_av == 1 & dlbike_av == 1 & everything else == 0
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
    #'scooter'_av ==1 & 'dk_bike'_av == 1 & everything else == 0
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
    #'dk_bike_av' == 1 & 'dl_bike_av' == 1 & everything else == 0
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['scooter_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0
    )) | ((
# 2-5 miles
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 0) & (
    micro_pool_socio['dkbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 1
)))

micro_0_to_5 = micro_pool_socio.loc[micro0to5]

In [7]:
database4 = db.Database('micro_0_to_5', micro_0_to_5)
database4.panel("who") # remember to sort data by individual
globals().update(database4.variables)

C:\Users\rs49742\anaconda3\lib\site-packages\biogeme\tools.py:256: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_biogroups'] = (df[column] != df[column].shift(1)).cumsum()
C:\Users\rs49742\anaconda3\lib\site-packages\biogeme\tools.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['_biogroups'], inplace=True)


In [18]:
ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 1)
ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')

# B_SCOOTERTIME = Beta('B_SCOOTERTIME', 0, None, None, 0)
# B_SCOOTERTIME_S = Beta('B_SCOOTERTIME_S', 1, None, None, 0)
# B_SCOOTERTIME_RND = -exp(B_SCOOTERTIME + B_SCOOTERTIME_S*bioDraws('B_SCOOTERTIME_RND', 'HALTON'))
# B_BIKETIME = Beta('B_BIKETIME', 0, None, None, 0)
# # B_DKBIKETIME = Beta('B_DKBIKETIME', 0, None, None, 0)
# B_SCTRANSITTIME = Beta('B_SCTRANSITTIME', 0, None, None, 0)

B_TIME = Beta('B_TIME', 0, None, None, 0)
# B_TIME_S = Beta('B_TIME_S', 1, None, None, 0)
# B_TIME_RND = -exp(B_TIME + B_TIME_S * bioDraws('B_TIME_RND', 'NORMAL_HALTON2'))

B_COST = Beta('B_COST', 0, None, None, 0)
# B_COST_S = Beta('B_COST_S', 1, None, None, 0)
# B_COST_RND = B_COST + B_COST_S * bioDraws('B_COST_RND', 'NORMAL')

B_ACCESS = Beta('B_ACCESS', 0, None, None, 0)
B_DROP =Beta('B_DROP', 0, None, None, 0)
B_WAITAV = Beta('B_WAITAV', 0, None, None, 0)
B_AV = Beta('B_AV', 0, None, None, 0)

# B_costadj = Beta('B_costadj', 0, None, None, 0)

B_PRCP_Yes = Beta('B_PRCP_Yes', 0, None, None, 0)
B_OWNBIKE = Beta('B_OWNBIKE', 0, None, None, 0)
B_BIKELANE = Beta('B_BIKELANE', 0, None, None, 0)
B_AGE_BIKE = Beta('B_AGE_BIKE', 0, None, None, 0)
B_AGE_SQ_BIKE = Beta('B_AGE_SQ_BIKE', 0, None, None, 0)
                    
B_AGE_SCTRANSIT = Beta('B_AGE_SCTRANSIT', 0, None, None, 0)
B_AGE_SQ_SCTRANSIT = Beta('B_AGE_SQ_SCTRANSIT', 0, None, None, 0)
                   
B_GENDER_F_BIKE = Beta('B_GENDER_F_BIKE', 0, None, None, 0)
B_GENDER_F_SCTRANSIT = Beta('B_GENDER_F_SCTRANSIT', 0, None, None, 0)

In [25]:
# Utility Functions
V7 = (ASC_SCOOTER_RND 
      + B_TIME*sctime
      + B_COST*sccost_adj
      + B_ACCESS*SCAW*(1-AVtech)
      + B_WAITAV*SCAV*AVtech 
      )
V8 = (ASC_DLBIKE_RND
        + B_TIME*dltime
        + B_COST*dlcost_adj
        + B_ACCESS*DLAW
       # + B_PRCP_Yes*PRCP_Yes
        + B_OWNBIKE*bike
       # + B_BIKELANE*BKLN_Yes
       # + B_AGE_SQ_BIKE*age*age + B_AGE_BIKE*age
       # + B_GENDER_F_BIKE*gender_F
    )
V9 = (ASC_DKBIKE_RND 
      + B_TIME*dbtime 
      + B_COST*dbcost_adj
      + B_ACCESS*DBAW
      + B_DROP*DBDW
      + B_OWNBIKE*bike
      # + B_BIKELANE*BKLN_Yes
      # + B_AGE_SQ_BIKE*age*age + B_AGE_BIKE*age
      # + B_GENDER_F_BIKE*gender_F
    )
V10 = (ASC_SCTRANSIT_RND
       + B_TIME*sttotime
        + B_COST*sttocost_adj 
        + B_ACCESS*STAW*(1-AVtech)
        + B_WAITAV*STAV*AVtech 
        # + B_AGE_SQ_SCTRANSIT*age*age + B_AGE_SCTRANSIT*age
        # + B_GENDER_F_SCTRANSIT*gender_F
    )

In [ ]:
V = {7: V7, 8: V8, 9: V9, 10: V10}
av = {7: scooter_av, 8: dlbike_av, 9: dkbike_av, 10: sctransit_av}

prob = models.logit(V, av, choice)
condprobIndiv = PanelLikelihoodTrajectory(prob)
logprob = log(MonteCarlo(condprobIndiv))
biogeme = bio.BIOGEME(database4, logprob, numberOfDraws = 5000)
biogeme.modelName = "micromobility_panel_mixedlogit_0_to_5"

results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults.round(2))

[12:14:16] < Warning >   Cannot read file __micromobility_panel_mixedlogit_0_to_5.iter. Statement is ignored.
